In [4]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from math import gcd
from numpy.random import randint
import numpy as np

def qft_dagger(n):
    """n-qubit QFT†."""
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    return qc

def c_amod15(a, power):
    """Controlled multiplication by a mod 15."""
    U = QuantumCircuit(4)
    for iteration in range(power):
        U.swap(2,3)
        U.swap(1,2)
        U.swap(0,1)
        for q in range(4):
            U.x(q)
    U = U.to_gate()
    U.name = f"{a}^{power} mod 15"
    return U

def shor_circuit(N):
    n = N.bit_length()
    a = randint(2, N-1)
    if gcd(a, N) != 1:
        print(f"Found factor by luck: {gcd(a, N)}")
        return None
    
    # Increase the number of qubits
    qc = QuantumCircuit(2*n+4, n)
    
    # Initialize counting qubits in superposition
    for q in range(2*n):
        qc.h(q)
    
    # Initialize auxiliary qubits
    qc.x(2*n)
    
    # Apply controlled U operations
    for i in range(2*n):
        qc.append(c_amod15(a, 2**i).control(1), [i] + list(range(2*n, 2*n+4)))
    
    # Apply inverse QFT
    qc.append(qft_dagger(2*n), range(2*n))
    
    # Measure
    qc.measure(range(n), range(n))
    
    return qc

# Example usage
N = 15  # Number to factorize
qc = shor_circuit(N)

if qc:
    simulator = AerSimulator()
    transpiled_qc = transpile(qc, simulator)
    job = simulator.run(transpiled_qc, shots=1000)
    result = job.result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()
    print(qc)
else:
    print("Circuit not created due to lucky factorization.")

      ┌───┐                                                            »
 q_0: ┤ H ├───────■────────────────────────────────────────────────────»
      ├───┤       │                                                    »
 q_1: ┤ H ├───────┼──────────────■─────────────────────────────────────»
      ├───┤       │              │                                     »
 q_2: ┤ H ├───────┼──────────────┼──────────────■──────────────────────»
      ├───┤       │              │              │                      »
 q_3: ┤ H ├───────┼──────────────┼──────────────┼──────────────■───────»
      ├───┤       │              │              │              │       »
 q_4: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼───────»
      ├───┤       │              │              │              │       »
 q_5: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼───────»
      ├───┤       │              │              │              │       »
 q_6: ┤ H ├───────┼──────────────┼──────────────┼──